In [ ]:

# 4.2. Entrenamiento
print("\n--- COMENZANDO ENTRENAMIENTO DE LA RED NEURONAL ---")
history = model.fit(
    x=[X_cuadrante_train, X_num_train],
    y=Y_train,
    validation_data=([X_cuadrante_val, X_num_val], Y_val),
    epochs=50, 
    batch_size=32,
    verbose=1
)
print("✅ Entrenamiento del Modelo de Red Neuronal Finalizado.")


# --- 5. PREPARACIÓN Y PREDICCIÓN FINAL (Ejemplo para Enero 2024) ---

# Usamos el set de prueba para demostrar la predicción
X_cuadrante_test = df_test['CUADRANTE_ID'].values.astype(np.int32)
X_num_test = df_test[features].values.astype(np.float32)

# 5.1. Predicción en escala normalizada
y_pred_scaled = model.predict([X_cuadrante_test, X_num_test])

# 5.2. Desnormalizar la predicción
# Se crea un array 2D para inverse_transform
y_pred_denormalized = scaler_robos_total.inverse_transform(y_pred_scaled)

# 5.3. Formatear las 78 Salidas
df_results = pd.DataFrame({
    'CUADRANTE_ID': df_test['CUADRANTE_ID'].values,
    'FECHA': df_test['FECHA'].values,
    'PREDICCION_ROBOS_REAL': y_pred_denormalized.flatten()
})



Total de filas listas para modelar: 8424

--- COMENZANDO ENTRENAMIENTO DE LA RED NEURONAL ---
Epoch 1/50


/usr/local/python/3.12.1/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0101 - mae: 0.0714 - val_loss: 0.0074 - val_mae: 0.0641
Epoch 2/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0101 - mae: 0.0714 - val_loss: 0.0074 - val_mae: 0.0641
Epoch 2/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0085 - mae: 0.0651 - val_loss: 0.0072 - val_mae: 0.0622
Epoch 3/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0085 - mae: 0.0651 - val_loss: 0.0072 - val_mae: 0.0622
Epoch 3/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0082 - mae: 0.0637 - val_loss: 0.0077 - val_mae: 0.0654
Epoch 4/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0082 - mae: 0.0637 - val_loss: 0.0077 - val_mae: 0.0654
Epoch 4/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0080 - mae: 0.0628 - val_loss: 0.0068 - val_mae: 0.0604
Epoch 5/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0080 - mae: 0.0628 - val_loss: 0.0068 - val_mae: 0.0604
Epoch 5/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0079 

In [41]:
# Filtrar un mes específico y ordenar (ej: Enero 2024)
MES_A_EVALUAR = 1
prediccion_enero = df_results[df_results['FECHA'].dt.month == MES_A_EVALUAR].copy()
prediccion_enero['PREDICCION_ROBOS_REAL'] = prediccion_enero['PREDICCION_ROBOS_REAL'].round(0).astype(int)
prediccion_enero = prediccion_enero.sort_values(by='PREDICCION_ROBOS_REAL', ascending=False)


print(f"\n--- TOP 10 CUADRANTES CON MAYOR INCIDENCIA (Predicción para Enero 2024) ---")
print(prediccion_enero[['CUADRANTE_ID', 'PREDICCION_ROBOS_REAL']].head(10))


--- TOP 10 CUADRANTES CON MAYOR INCIDENCIA (Predicción para Enero 2024) ---
     CUADRANTE_ID  PREDICCION_ROBOS_REAL
0               1                      0
12              2                      0
24              3                      0
36              4                      0
48              5                      0
60              6                      0
72              7                      0
84              8                      0
96              9                      0
108            10                      0


In [ ]:
import pandas as pd
import numpy as np

# --- 1. Carga y Transformación de Formato Ancho a Largo ---

file_name = "cuadrantes_negocios.csv"
df_wide = pd.read_csv(file_name)


# Renombrar 'CUADRANTE' a 'CUADRANTE_ID' y 'POBLACION'
df_wide.rename(columns={'CUADRANTE': 'CUADRANTE_ID', 'POBLACION': 'POBLACION_BASE'}, inplace=True)

# Identificar las columnas de robos (todas las columnas excepto 'CUADRANTE_ID' y 'POBLACION_BASE')
id_vars = ['CUADRANTE_ID', 'POBLACION_BASE']
value_vars = [col for col in df_wide.columns if col not in id_vars]

# Transformación de Formato Ancho a Largo (Melting)
# Esto crea una fila por cada combinación de Cuadrante y Mes/Año
df_long = df_wide.melt(
    id_vars=id_vars,
    value_vars=value_vars,
    var_name='MES_AÑO_STR', # Columna temporal que contiene el mes y el año
    value_name='ROBOS_TOTAL' # La variable objetivo
).copy()

# 1.1. Parsear la columna MES_AÑO_STR para crear la columna de FECHA
# Ejemplo: 'ROBOS A NEGOCIOS MES 1 2015' -> Enero 2015
def parse_date(date_str):
    # Separar la cadena por espacios
    parts = date_str.split(' ')
    mes = parts[4] # Ej: '1'
    año = parts[5] # Ej: '2015'
    return f'{año}-{mes}-01'

df_long['FECHA'] = df_long['MES_AÑO_STR'].apply(parse_date)
df_long['FECHA'] = pd.to_datetime(df_long['FECHA'])
df_long.sort_values(by=['CUADRANTE_ID', 'FECHA'], inplace=True)
df_long.drop(columns=['MES_AÑO_STR'], inplace=True)


# --- 2. Generación de Características Temporales ---

df_long['AÑO'] = df_long['FECHA'].dt.year
df_long['MES'] = df_long['FECHA'].dt.month
# df_long['DIA_DEL_MES'] = df_long['FECHA'].dt.day # Si fuera relevante, pero para mensual, el mes ya lo captura

# --- 3. Generación de Características Históricas (Retardos) ---

# Agrupar por CUADRANTE_ID para asegurar que el retardo sea solo dentro de esa serie de tiempo.
df_long['ROBOS_t-1'] = df_long.groupby('CUADRANTE_ID')['ROBOS_TOTAL'].shift(1)
df_long['ROBOS_t-12'] = df_long.groupby('CUADRANTE_ID')['ROBOS_TOTAL'].shift(12)

# Generación de la Media Móvil de 3 meses (excluyendo el mes actual)
# Usamos un Rolling Window de 3 periodos y lo shift(1) para aplicarlo al mes siguiente.
df_long['MEDIA_MOVIL_3M'] = df_long.groupby('CUADRANTE_ID')['ROBOS_TOTAL'].rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True).shift(1)


# --- 4. Limpieza Final para el Modelo ---

# Eliminar las filas que tienen valores NaN en los retardos (los primeros 12 meses de cada cuadrante)
# Estos no tienen suficiente historial para las variables t-12.
df_model_input = df_long.dropna(subset=['ROBOS_t-12']).copy()

# Seleccionar solo las columnas que servirán como entradas (X) y la salida (Y)
# NOTA: La POBLACION_BASE es estática. Otras variables como la violencia se añadirían aquí.
columnas_finales = [
    'CUADRANTE_ID',
    'FECHA',
    'AÑO',
    'MES',
    'POBLACION_BASE',
    'ROBOS_t-1',
    'ROBOS_t-12',
    'MEDIA_MOVIL_3M',
    'ROBOS_TOTAL' # Esta será la variable objetivo (Y)
]

df_model_input = df_model_input[columnas_finales]

print("--- Vista Preliminar de la Tabla de Entradas (Formato Largo) ---")
print(f"Total de filas generadas para el modelo: {len(df_model_input)}")
print("Ejemplo de las entradas para el CUADRANTE 1:")
print(df_model_input[df_model_input['CUADRANTE_ID'] == 1].head(15))

In [ ]:
# exportar df_model_input a un archivo csv
df_model_input.to_csv("ficosec_model_input.csv", index=False)

In [ ]:
# exportar df_model_input a un archivo xlsx
df_model_input.to_excel("ficosec_model_input.xlsx", index=False)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Suponiendo que 'df_model_input' es el DataFrame generado en el paso anterior

# 1. Identificar las columnas numéricas que necesitan escalado
features_to_scale = [
    'POBLACION_BASE',
    'ROBOS_t-1',
    'ROBOS_t-12',
    'MEDIA_MOVIL_3M',
    # La variable objetivo (Y) también debe escalarse
    'ROBOS_TOTAL' 
]

# 2. Inicializar el escalador
scaler = MinMaxScaler()

# 3. Aplicar el escalador a las columnas seleccionadas
df_model_input[features_to_scale] = scaler.fit_transform(df_model_input[features_to_scale])

# Guardar los escaladores si planeas desnormalizar las predicciones más tarde
# Por ejemplo, el scaler de ROBOS_TOTAL es crucial para llevar la predicción de vuelta a conteos reales.
scaler_robos_total = MinMaxScaler()
scaler_robos_total.fit(df_model_input[['ROBOS_TOTAL']]) # Se debe re-entrenar solo en el target si quieres desnormalizar fácilmente

print("✅ Normalización Min-Max aplicada a las variables numéricas.")
print(df_model_input[features_to_scale].head())

In [ ]:
# --- A. Codificación One-Hot para el MES ---

# Crear variables dummy para la columna 'MES'
df_processed = pd.get_dummies(df_model_input, columns=['MES'], prefix='MES')

# El CUADRANTE_ID se mantiene como está para el Embedding (se usará directamente en Keras/TensorFlow)

print("\n✅ Codificación One-Hot aplicada al MES.")
print(df_processed[[col for col in df_processed.columns if 'MES_' in col]].head())

In [ ]:
# 1. Definir las fechas de corte
FECHA_CORTE_VAL = pd.to_datetime('2023-01-01')
FECHA_CORTE_TEST = pd.to_datetime('2024-01-01')

# 2. Dividir los datos
df_train = df_processed[df_processed['FECHA'] < FECHA_CORTE_VAL].copy()
df_val = df_processed[(df_processed['FECHA'] >= FECHA_CORTE_VAL) & (df_processed['FECHA'] < FECHA_CORTE_TEST)].copy()
df_test = df_processed[df_processed['FECHA'] >= FECHA_CORTE_TEST].copy()

# 3. Definición de Entradas (X) y Salidas (Y)
features = [
    'POBLACION_BASE', 
    'ROBOS_t-1', 
    'ROBOS_t-12', 
    'MEDIA_MOVIL_3M'
] + [col for col in df_processed.columns if 'MES_' in col]

X_train = df_train[features]
Y_train = df_train['ROBOS_TOTAL']

X_val = df_val[features]
Y_val = df_val['ROBOS_TOTAL']

X_test = df_test[features]
Y_test = df_test['ROBOS_TOTAL']

# El CUADRANTE_ID se manejará como una entrada separada en el modelo de Keras/TensorFlow
train_cuadrante_id = df_train['CUADRANTE_ID']
val_cuadrante_id = df_val['CUADRANTE_ID']
test_cuadrante_id = df_test['CUADRANTE_ID']

print(f"\n✅ División de datos completada:")
print(f"  Entrenamiento (Train): {len(X_train)} filas")
print(f"  Validación (Validation): {len(X_val)} filas")
print(f"  Prueba (Test): {len(X_test)} filas")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate, Flatten

# --- CONSTANTES ---
NUM_CUADRANTES = df_model_input['CUADRANTE_ID'].nunique() # Debería ser 78
# La dimensión del embedding debe ser menor que el número de categorías. 
# Una regla empírica simple es tomar la raíz cuadrada del número de categorías.
EMBEDDING_DIM = int(np.sqrt(NUM_CUADRANTES)) # Ej: sqrt(78) ~ 8 o 9. Usemos 10.
EMBEDDING_DIM = 10
NUM_FEAT_NUMERICAS = X_train.shape[1] 

# --- CONVERSIÓN A ARRAYS DE NUMPY ---

# 1. Entradas para el Embedding (CUADRANTE_ID)
# Keras requiere que los IDs empiecen en 0 o 1. Usaremos 1 a N.
# NOTA: Los DataFrames 'train_cuadrante_id', 'val_cuadrante_id', 'test_cuadrante_id'
# ya están preparados para esto (valores de 1 a 78).
X_cuadrante_train = train_cuadrante_id.values
X_cuadrante_val = val_cuadrante_id.values
X_cuadrante_test = test_cuadrante_id.values

# 2. Entradas para las Características Numéricas (Robos t-1, Mes_1, etc.)
X_num_train = X_train.values
X_num_val = X_val.values
X_num_test = X_test.values

# 3. Salidas (Objetivo)
Y_train = Y_train.values
Y_val = Y_val.values
Y_test = Y_test.values

print(f"Número de Cuadrantes (Categorías): {NUM_CUADRANTES}")
print(f"Dimensión del Vector de Embedding: {EMBEDDING_DIM}")
print(f"Número de Características Numéricas/One-Hot: {NUM_FEAT_NUMERICAS}")

In [ ]:
# --- 1. Rama de Entrada para el CUADRANTE_ID (Embedding Layer) ---

# Input: Un array de números enteros (IDs de 1 a 78)
input_cuadrante = Input(shape=(1,), name='input_cuadrante')

# Embedding: Convierte cada ID de cuadrante en un vector denso (ej: 78x10)
# input_dim: Rango de los IDs (78 cuadrantes + 1 por convención)
# output_dim: La dimensión de nuestro vector de embedding (10)
embedding_layer = Embedding(
    input_dim=NUM_CUADRANTES + 1,
    output_dim=EMBEDDING_DIM,
    input_length=1,
    name='embedding_cuadrante'
)(input_cuadrante)

# Aplanar el Embedding (de (None, 1, 10) a (None, 10))
flatten_embedding = Flatten()(embedding_layer)


# --- 2. Rama de Entrada para las Características Numéricas ---

# Input: Un array con todas las características (Robos t-1, Mes_1, Población, etc.)
input_numerico = Input(shape=(NUM_FEAT_NUMERICAS,), name='input_numerico')


# --- 3. Combinación de Ramas (Merging) ---

# Concatenar el vector de embedding y las características numéricas
combined = Concatenate()([flatten_embedding, input_numerico])

# Capas Densa de la Red Neuronal (Deep Layers)
# Se utilizan capas profundas para aprender las interacciones complejas.
dense_1 = Dense(64, activation='relu')(combined)
dense_2 = Dense(32, activation='relu')(dense_1)

# Capa de Salida
# Output: Un único nodo que predice el valor escalado de ROBOS_TOTAL (Y).
output_layer = Dense(1, activation='linear', name='prediccion_final')(dense_2)


# --- 4. Crear el Modelo Final ---

model = Model(inputs=[input_cuadrante, input_numerico], outputs=output_layer)

# Compilación: Usamos 'mean_squared_error' (MSE) para regresión y Adam como optimizador.
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

print(model.summary())

In [ ]:
# --- CÓDIGO DE VERIFICACIÓN Y CORRECCIÓN DE TIPOS (Solución al ValueError) ---

# 1. Identificar todas las columnas que deberían ser numéricas (inputs y targets)
numerical_and_target_cols = [
    'POBLACION_BASE',
    'ROBOS_t-1',
    'ROBOS_t-12',
    'MEDIA_MOVIL_3M',
    'ROBOS_TOTAL'
]

# 2. Forzar la conversión de estas columnas a numérico (float)
# El argumento 'coerce' convierte cualquier valor no numérico (texto) a NaN.
for col in numerical_and_target_cols:
    df_model_input[col] = pd.to_numeric(df_model_input[col], errors='coerce')

# 3. Eliminar o Rellenar los valores NaN que se generaron
# Es más seguro eliminar las filas donde esto ocurre, aunque deberían ser pocas.
# Si el error ocurrió en una fila crítica (ej. ROBOS_TOTAL), la fila es inutilizable.
filas_antes = len(df_model_input)
df_model_input.dropna(subset=numerical_and_target_cols, inplace=True)
filas_despues = len(df_model_input)

if filas_antes != filas_despues:
    print(f"⚠️ Alerta: Se eliminaron {filas_antes - filas_despues} filas debido a valores no numéricos ocultos.")

# 4. Asegurar que las columnas de ID sean Enteros (para el Embedding)
# El Embedding Layer requiere IDs enteros.
df_model_input['CUADRANTE_ID'] = df_model_input['CUADRANTE_ID'].astype(np.int32)
df_model_input['MES'] = df_model_input['MES'].astype(np.int32) 

print("\n✅ Verificación de tipos de datos completada. Todos los inputs/outputs numéricos están limpios.")

In [ ]:
print(X_num_train.dtype)

In [ ]:
# convertir X_num_train a float32
X_num_train = X_num_train.astype(np.int32)
X_num_test = X_num_test.astype(np.int32)

In [ ]:
print(X_cuadrante_train.dtype)

In [ ]:
# convertir X_cuadrante_train a float64
X_cuadrante_train = X_cuadrante_train.astype(np.float64)
X_cuadrante_test = X_cuadrante_test.astype(np.float64)

In [ ]:
# Configuración del entrenamiento
history = model.fit(
    x=[X_cuadrante_train, X_num_train],
    y=Y_train,
    validation_data=([X_cuadrante_val, X_num_val], Y_val),
    epochs=50, # Puedes empezar con 50 o 100 épocas
    batch_size=32,
    verbose=1
)

print("\n✅ Entrenamiento del Modelo de Red Neuronal Finalizado.")

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate, Flatten
from tensorflow.keras.losses import Poisson
import warnings
warnings.filterwarnings('ignore')

# --- CONFIGURACIÓN Y CONSTANTES ---
# Asegúrate de que este sea el nombre de tu archivo
FILE_NAME = "cuadrantes_negocios.csv" 
NUM_CUADRANTES = 78
EMBEDDING_DIM = 10 
EPOCHS = 100
BATCH_SIZE = 16

# --- 1. CARGA, TRANSFORMACIÓN (ANCHO A LARGO) Y GENERACIÓN DE FEATURES ---

def load_and_prepare_data(file_name):
    """Carga, transforma el formato ancho a largo y genera features de retardo."""
    try:
        df_wide = pd.read_csv(file_name)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo '{file_name}'.")
        raise

    df_wide.rename(columns={'CUADRANTE': 'CUADRANTE_ID', 'POBLACION': 'POBLACION_BASE'}, inplace=True)
    
    id_vars = ['CUADRANTE_ID', 'POBLACION_BASE']
    value_vars = [col for col in df_wide.columns if col not in id_vars]

    # Transformación de Ancho a Largo (Melt)
    df_long = df_wide.melt(
        id_vars=id_vars,
        value_vars=value_vars,
        var_name='MES_AÑO_STR',
        value_name='ROBOS_TOTAL'
    ).copy()

    # Parsear la fecha
    def parse_date(date_str):
        parts = date_str.split(' ')
        # Asumiendo el formato 'ROBOS A NEGOCIOS MES 1 2015'
        mes = parts[4] 
        año = parts[5] 
        return f'{año}-{mes}-01'

    df_long['FECHA'] = df_long['MES_AÑO_STR'].apply(parse_date)
    df_long['FECHA'] = pd.to_datetime(df_long['FECHA'])
    df_long.sort_values(by=['CUADRANTE_ID', 'FECHA'], inplace=True)
    df_long.drop(columns=['MES_AÑO_STR'], inplace=True)

    # Generación de Características Temporales y Retardos
    df_long['AÑO'] = df_long['FECHA'].dt.year
    df_long['MES'] = df_long['FECHA'].dt.month
    df_long['ROBOS_t-1'] = df_long.groupby('CUADRANTE_ID')['ROBOS_TOTAL'].shift(1)
    df_long['ROBOS_t-12'] = df_long.groupby('CUADRANTE_ID')['ROBOS_TOTAL'].shift(12)
    df_long['MEDIA_MOVIL_3M'] = df_long.groupby('CUADRANTE_ID')['ROBOS_TOTAL'].rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True).shift(1)
    
    # Eliminar filas con NaN en los retardos (primeros 12 meses)
    df_model_input = df_long.dropna(subset=['ROBOS_t-12']).copy()
    
    return df_model_input

# Cargar y preparar los datos
df_model_input = load_and_prepare_data(FILE_NAME)


# --- 2. LIMPIEZA, NORMALIZACIÓN (SOLO FEATURES) Y CODIFICACIÓN ---

# 2.1. Verificación y Corrección de Tipos (Limpieza de "object")
numerical_and_target_cols = ['POBLACION_BASE', 'ROBOS_t-1', 'ROBOS_t-12', 'MEDIA_MOVIL_3M', 'ROBOS_TOTAL']
for col in numerical_and_target_cols:
    df_model_input[col] = pd.to_numeric(df_model_input[col], errors='coerce')

df_model_input.dropna(subset=numerical_and_target_cols, inplace=True)
df_model_input['CUADRANTE_ID'] = df_model_input['CUADRANTE_ID'].astype(np.int32)
df_model_input['MES'] = df_model_input['MES'].astype(np.int32)

# 2.2. Normalización Min-Max (¡SOLO VARIABLES DE ENTRADA/FEATURES!)
scaler = MinMaxScaler()
features_to_scale = ['POBLACION_BASE', 'ROBOS_t-1', 'ROBOS_t-12', 'MEDIA_MOVIL_3M']

# El target 'ROBOS_TOTAL' NO se normaliza para la Pérdida de Poisson.
df_model_input[features_to_scale] = scaler.fit_transform(df_model_input[features_to_scale])

# 2.3. Codificación One-Hot para el MES
df_processed = pd.get_dummies(df_model_input, columns=['MES'], prefix='MES')


# --- 3. DIVISIÓN DE DATOS Y PREPARACIÓN DE ARRAYS ---

FECHA_CORTE_VAL = pd.to_datetime('2023-01-01')
FECHA_CORTE_TEST = pd.to_datetime('2024-01-01')

# División
df_train = df_processed[df_processed['FECHA'] < FECHA_CORTE_VAL]
df_val = df_processed[(df_processed['FECHA'] >= FECHA_CORTE_VAL) & (df_processed['FECHA'] < FECHA_CORTE_TEST)]
df_test = df_processed[df_processed['FECHA'] >= FECHA_CORTE_TEST]

# Definición de Features para la rama numérica
features = features_to_scale + [col for col in df_processed.columns if 'MES_' in col]


# Conversión a Arrays de NumPy con tipo explícito
X_cuadrante_train = df_train['CUADRANTE_ID'].values.astype(np.int32)
X_cuadrante_val = df_val['CUADRANTE_ID'].values.astype(np.int32)
X_cuadrante_test = df_test['CUADRANTE_ID'].values.astype(np.int32)

X_num_train = df_train[features].values.astype(np.float32)
X_num_val = df_val[features].values.astype(np.float32)
X_num_test = df_test[features].values.astype(np.float32)

# Y_train usa el valor de conteo original (CORRECCIÓN CRÍTICA)
Y_train = df_train['ROBOS_TOTAL'].values.astype(np.float32)
Y_val = df_val['ROBOS_TOTAL'].values.astype(np.float32)
Y_test = df_test['ROBOS_TOTAL'].values.astype(np.float32)


# --- 4. CONSTRUCCIÓN Y ENTRENAMIENTO DEL MODELO DE RED NEURONAL ---

def build_model(num_cuadrantes, embedding_dim, num_feat_numericas):
    
    # 1. Rama Embedding
    input_cuadrante = Input(shape=(1,), name='input_cuadrante')
    embedding_layer = Embedding(
        input_dim=num_cuadrantes + 1, 
        output_dim=embedding_dim,
        input_length=1,
        name='embedding_cuadrante'
    )(input_cuadrante)
    flatten_embedding = Flatten()(embedding_layer)

    # 2. Rama Numérica
    input_numerico = Input(shape=(num_feat_numericas,), name='input_numerico')

    # 3. Concatenación y Capas Profundas (AUMENTADAS)
    combined = Concatenate()([flatten_embedding, input_numerico])
    
    # Capas Ocultas con mayor capacidad
    dense_1 = Dense(128, activation='relu')(combined) 
    dense_2 = Dense(64, activation='relu')(dense_1)   
    dense_3 = Dense(32, activation='relu')(dense_2)   
    
    # 4. Capa de Salida: SOFTPLUS para asegurar predicciones NO NEGATIVAS.
    output_layer = Dense(1, activation='softplus', name='prediccion_final')(dense_3)

    model = Model(inputs=[input_cuadrante, input_numerico], outputs=output_layer)
    
    # Compilación: LOSS POISSON para datos de conteo.
    model.compile(optimizer='adam', loss=Poisson(), metrics=['mae'])
    
    return model

model = build_model(NUM_CUADRANTES, EMBEDDING_DIM, X_num_train.shape[1])

# 4.2. Entrenamiento
print("\n--- COMENZANDO ENTRENAMIENTO (Poisson Loss) ---")
print("El modelo ahora entrena con valores de conteo originales.")

history = model.fit(
    x=[X_cuadrante_train, X_num_train],
    y=Y_train,
    validation_data=([X_cuadrante_val, X_num_val], Y_val),
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE, 
    verbose=1
)
print("✅ Entrenamiento del Modelo de Red Neuronal Finalizado.")


# --- 5. PREDICCIÓN FINAL (Ejemplo para Enero 2024) ---

# 5.1. Predicción: El resultado ya está en la escala de conteo real.
y_pred_real = model.predict([X_cuadrante_test, X_num_test])

# 5.2. Formatear las 78 Salidas
df_results = pd.DataFrame({
    'CUADRANTE_ID': df_test['CUADRANTE_ID'].values,
    'FECHA': df_test['FECHA'].values,
    # La predicción ya es el conteo real, solo se redondea.
    'PREDICCION_ROBOS_REAL': y_pred_real.flatten()
})


--- COMENZANDO ENTRENAMIENTO (Poisson Loss) ---
El modelo ahora entrena con valores de conteo originales.
Epoch 1/100
410/410 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.4836 - mae: 1.1007 - val_loss: 0.6427 - val_mae: 0.9521
Epoch 2/100
410/410 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3688 - mae: 1.0077 - val_loss: 0.6110 - val_mae: 0.9030
Epoch 3/100
410/410 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3564 - mae: 0.9962 - val_loss: 0.6502 - val_mae: 0.9847
Epoch 4/100
410/410 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3544 - mae: 0.9935 - val_loss: 0.6385 - val_mae: 0.9580
Epoch 5/100
410/410 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3518 - mae: 0.9954 - val_loss: 0.6204 - val_mae: 0.9294
Epoch 6/100
410/410 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3502 - mae: 0.9906 - val_loss: 0.6255 - val_mae: 0.9313
Epoch 7/100
410/410 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3451 - mae: 0.9853 - val_loss: 0.6341 - val_mae: 0.9481
Epoch 8/100
410/410 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss:

In [5]:
# Filtrar un mes específico y ordenar (ej: Enero 2024)
MES_A_EVALUAR = 1
prediccion_enero = df_results[df_results['FECHA'].dt.month == MES_A_EVALUAR].copy()
prediccion_enero['PREDICCION_ROBOS_REAL'] = prediccion_enero['PREDICCION_ROBOS_REAL'].round(0).astype(int)
prediccion_enero = prediccion_enero.sort_values(by='PREDICCION_ROBOS_REAL', ascending=False)


print(f"\n--- TOP 10 CUADRANTES CON MAYOR INCIDENCIA (Predicción para Enero 2024) ---")
print(prediccion_enero[['CUADRANTE_ID', 'PREDICCION_ROBOS_REAL']].head(10))


--- TOP 10 CUADRANTES CON MAYOR INCIDENCIA (Predicción para Enero 2024) ---
     CUADRANTE_ID  PREDICCION_ROBOS_REAL
432            37                      6
288            25                      6
144            13                      5
816            69                      5
252            22                      4
648            55                      4
828            70                      4
360            31                      3
324            28                      3
336            29                      3


In [9]:
import pandas as pd
import numpy as np

# --- 1. Preparación de los datos reales (Y_test) ---

# Aseguramos que Y_test use el valor de conteo original.
# Nota: Y_test se definió como df_test['ROBOS_TOTAL'].values.astype(np.float32)
# Esto se hizo antes del entrenamiento, por lo que ya contiene los valores REALES.

# Creamos una columna con los valores reales del conjunto de prueba.
df_test['ROBOS_REAL'] = Y_test 


# --- 2. Formato de la Tabla de Resultados ---

# df_results fue generado en el paso anterior y contiene:
# ['CUADRANTE_ID', 'FECHA', 'PREDICCION_ROBOS_REAL']

# Unimos los resultados de la predicción con los valores reales
df_comparacion = df_results.merge(
    df_test[['CUADRANTE_ID', 'FECHA', 'ROBOS_REAL']], 
    on=['CUADRANTE_ID', 'FECHA'], 
    how='left'
).copy()

# Redondear y asegurar que los conteos sean enteros
df_comparacion['PREDICCION_ROBOS_REAL'] = df_comparacion['PREDICCION_ROBOS_REAL'].round(0).astype(int)
df_comparacion['ROBOS_REAL'] = df_comparacion['ROBOS_REAL'].round(0).astype(int)

# --- 3. Filtrar para Enero de 2024 y Mostrar ---

MES_A_EVALUAR = 1
AÑO_A_EVALUAR = 2024

df_enero_2024 = df_comparacion[
    (df_comparacion['FECHA'].dt.month == MES_A_EVALUAR) & 
    (df_comparacion['FECHA'].dt.year == AÑO_A_EVALUAR)
].sort_values(by='PREDICCION_ROBOS_REAL', ascending=False).reset_index(drop=True)

# Añadir la diferencia (Error)
df_enero_2024['ERROR (Real - Pred)'] = df_enero_2024['ROBOS_REAL'] - df_enero_2024['PREDICCION_ROBOS_REAL']

# Seleccionar y mostrar las columnas clave
df_final = df_enero_2024[['CUADRANTE_ID', 'ROBOS_REAL', 'PREDICCION_ROBOS_REAL', 'ERROR (Real - Pred)']]

print(f"\n--- Comparación de Robos a Negocio: {MES_A_EVALUAR}/{AÑO_A_EVALUAR} ---")
print("Ordenado por la Predicción (Mayor Riesgo)")

# 

print(df_final.head(20).to_markdown(index=False))


--- Comparación de Robos a Negocio: 1/2024 ---
Ordenado por la Predicción (Mayor Riesgo)
|   CUADRANTE_ID |   ROBOS_REAL |   PREDICCION_ROBOS_REAL |   ERROR (Real - Pred) |
|---------------:|-------------:|------------------------:|----------------------:|
|             37 |            3 |                       6 |                    -3 |
|             25 |            2 |                       6 |                    -4 |
|             13 |            6 |                       5 |                     1 |
|             69 |            0 |                       5 |                    -5 |
|             22 |            5 |                       4 |                     1 |
|             55 |            8 |                       4 |                     4 |
|             70 |            4 |                       4 |                     0 |
|             31 |            2 |                       3 |                    -1 |
|             28 |            0 |                       3 |           

In [10]:
from sklearn.metrics import r2_score
import numpy as np

# --- 1. Obtener las predicciones del modelo ---
# Nota: Asumimos que X_cuadrante_test y X_num_test ya están definidos y listos.

# Predicción en escala de conteo (no normalizada)
y_pred_real = model.predict([X_cuadrante_test, X_num_test])

# --- 2. Preparar los datos para R2 ---

# Y_test es el valor real (conteo, float32)
# y_pred_real es el valor predicho (conteo, float32)

# Scikit-learn espera arrays 1D. Aplanamos las predicciones.
y_true = Y_test 
y_predicted = y_pred_real.flatten()

# Opcional: Redondear a entero para el R2, ya que son conteos (aunque no es estrictamente necesario)
# Usaremos el valor flotante para mayor precisión en la métrica:

# --- 3. Calcular el R^2 ---

# La función r2_score maneja el cálculo R^2
r2 = r2_score(y_true, y_predicted)

# --- 4. Mostrar el Resultado ---

print("\n--- Evaluación del Modelo de Red Neuronal ---")
print(f"Número total de predicciones evaluadas: {len(y_true)}")
print(f"El Coeficiente de Determinación (R^2) en el conjunto de prueba es: {r2:.4f}")

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 

--- Evaluación del Modelo de Red Neuronal ---
Número total de predicciones evaluadas: 936
El Coeficiente de Determinación (R^2) en el conjunto de prueba es: -0.1874


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate, Flatten, LSTM, Dropout
from tensorflow.keras.losses import Poisson
import warnings
warnings.filterwarnings('ignore') # Ocultar advertencias

# --- CONFIGURACIÓN Y CONSTANTES ---
FILE_NAME = "cuadrantes_negocios.csv"
NUM_CUADRANTES = 78
EMBEDDING_DIM = 10 
EPOCHS = 100
BATCH_SIZE = 16
TIME_STEPS = 3 # Ventana de tiempo (3 meses anteriores)

# --- 1. CARGA, TRANSFORMACIÓN (ANCHO A LARGO) Y GENERACIÓN DE FEATURES ---

def load_and_prepare_data(file_name):
    """Carga, transforma el formato ancho a largo y genera features de retardo."""
    try:
        df_wide = pd.read_csv(file_name)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo '{file_name}'.")
        raise

    df_wide.rename(columns={'CUADRANTE': 'CUADRANTE_ID', 'POBLACION': 'POBLACION_BASE'}, inplace=True)
    
    id_vars = ['CUADRANTE_ID', 'POBLACION_BASE']
    value_vars = [col for col in df_wide.columns if col not in id_vars]

    df_long = df_wide.melt(
        id_vars=id_vars,
        value_vars=value_vars,
        var_name='MES_AÑO_STR',
        value_name='ROBOS_TOTAL'
    ).copy()

    def parse_date(date_str):
        parts = date_str.split(' ')
        mes = parts[4] 
        año = parts[5] 
        return f'{año}-{mes}-01'

    df_long['FECHA'] = df_long['MES_AÑO_STR'].apply(parse_date)
    df_long['FECHA'] = pd.to_datetime(df_long['FECHA'])
    df_long.sort_values(by=['CUADRANTE_ID', 'FECHA'], inplace=True)
    df_long.drop(columns=['MES_AÑO_STR'], inplace=True)

    df_long['AÑO'] = df_long['FECHA'].dt.year
    df_long['MES'] = df_long['FECHA'].dt.month
    df_long['ROBOS_t-1'] = df_long.groupby('CUADRANTE_ID')['ROBOS_TOTAL'].shift(1)
    df_long['ROBOS_t-12'] = df_long.groupby('CUADRANTE_ID')['ROBOS_TOTAL'].shift(12)
    df_long['MEDIA_MOVIL_3M'] = df_long.groupby('CUADRANTE_ID')['ROBOS_TOTAL'].rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True).shift(1)
    
    df_model_input = df_long.dropna(subset=['ROBOS_t-12']).copy()
    
    return df_model_input

df_model_input = load_and_prepare_data(FILE_NAME)


# --- 2. LIMPIEZA, NORMALIZACIÓN (SOLO FEATURES) Y CODIFICACIÓN ---

numerical_and_target_cols = ['POBLACION_BASE', 'ROBOS_t-1', 'ROBOS_t-12', 'MEDIA_MOVIL_3M', 'ROBOS_TOTAL']
for col in numerical_and_target_cols:
    df_model_input[col] = pd.to_numeric(df_model_input[col], errors='coerce')

df_model_input.dropna(subset=numerical_and_target_cols, inplace=True)
df_model_input['CUADRANTE_ID'] = df_model_input['CUADRANTE_ID'].astype(np.int32)
df_model_input['MES'] = df_model_input['MES'].astype(np.int32)

# 2.2. Normalización Min-Max (SOLO FEATURES)
scaler = MinMaxScaler()
features_to_scale = ['POBLACION_BASE', 'ROBOS_t-1', 'ROBOS_t-12', 'MEDIA_MOVIL_3M']
df_model_input[features_to_scale] = scaler.fit_transform(df_model_input[features_to_scale])

# 2.3. Codificación One-Hot para el MES
df_processed = pd.get_dummies(df_model_input, columns=['MES'], prefix='MES')


# --- 3. DIVISIÓN DE DATOS Y PREPARACIÓN DE ARRAYS SEQUENCIALES ---

FECHA_CORTE_VAL = pd.to_datetime('2023-01-01')
FECHA_CORTE_TEST = pd.to_datetime('2024-01-01')

df_train = df_processed[df_processed['FECHA'] < FECHA_CORTE_VAL]
df_val = df_processed[(df_processed['FECHA'] >= FECHA_CORTE_VAL) & (df_processed['FECHA'] < FECHA_CORTE_TEST)]
df_test = df_processed[df_processed['FECHA'] >= FECHA_CORTE_TEST]

# Definición de Features para la LSTM
features_lstm = [col for col in df_processed.columns if col not in ['CUADRANTE_ID', 'FECHA', 'ROBOS_TOTAL', 'AÑO']]


def create_sequences(df, Y_values, time_steps):
    """Transforma datos tabulares en secuencias (Timesteps, Features) para LSTM."""
    Xs, Ys, Cuadrantes = [], [], []
    
    for id in df['CUADRANTE_ID'].unique():
        df_cuadrante = df[df['CUADRANTE_ID'] == id].copy().reset_index(drop=True)
        
        # Iteramos a través de la serie del tiempo para crear la ventana
        for i in range(len(df_cuadrante) - time_steps):
            
            # X_seq: Los 'time_steps' (3) meses anteriores para las features
            X_seq = df_cuadrante.loc[i:(i + time_steps - 1), features_lstm].values
            
            # Y_val: El valor a predecir (ROBOS_TOTAL) justo después de la ventana
            # Usamos el índice de df_cuadrante para obtener el valor real del target (Y)
            Y_val = df_cuadrante.loc[i + time_steps, 'ROBOS_TOTAL']
            
            # Cuadrante ID para el Embedding
            Cuadrante_ID_val = df_cuadrante.loc[i + time_steps, 'CUADRANTE_ID']
            
            Xs.append(X_seq)
            Ys.append(Y_val)
            Cuadrantes.append(Cuadrante_ID_val)

    # Conversión a arrays de NumPy con el tipo correcto
    return np.array(Xs, dtype=np.float32), np.array(Ys, dtype=np.float32), np.array(Cuadrantes, dtype=np.int32)


# Generar las nuevas entradas secuenciales
X_train_seq, Y_train_seq, X_cuadrante_train_seq = create_sequences(df_train, df_train['ROBOS_TOTAL'], TIME_STEPS)
X_val_seq, Y_val_seq, X_cuadrante_val_seq = create_sequences(df_val, df_val['ROBOS_TOTAL'], TIME_STEPS)
X_test_seq, Y_test_seq, X_cuadrante_test_seq = create_sequences(df_test, df_test['ROBOS_TOTAL'], TIME_STEPS)

# --- MODIFICACIÓN EN EL PASO 4 (build_lstm_model) ---
# [Solo mostraré la función build_lstm_model modificada]

def build_lstm_model(num_cuadrantes, embedding_dim, seq_length, num_features):
    
    # 1. Rama Embedding (CUADRANTE_ID)
    input_cuadrante = Input(shape=(1,), name='input_cuadrante')
    embedding_layer = Embedding(
        input_dim=num_cuadrantes + 1, 
        output_dim=embedding_dim,
        input_length=1,
        name='embedding_cuadrante'
    )(input_cuadrante)
    flatten_embedding = Flatten()(embedding_layer)

    # 2. Rama Secuencial (LSTM)
    input_sequence = Input(shape=(seq_length, num_features), name='input_sequence')
    
    # Capa LSTM 1: Aumentamos unidades y añadimos return_sequences=True
    lstm_out = LSTM(128, activation='tanh', return_sequences=True)(input_sequence) 
    lstm_out = Dropout(0.3)(lstm_out) # Aumentamos Dropout

    # Capa LSTM 2: Añadimos una segunda capa LSTM para profundidad
    lstm_out = LSTM(64, activation='tanh', return_sequences=False)(lstm_out)
    lstm_out = Dropout(0.3)(lstm_out)

    # 3. Concatenación y Capas Profundas
    combined = Concatenate()([flatten_embedding, lstm_out])
    
    # Capas Densa Finales (Aumentamos unidades)
    dense_1 = Dense(128, activation='relu')(combined)
    dense_2 = Dense(64, activation='relu')(dense_1)
    
    # 4. Capa de Salida: Softplus y Poisson Loss
    output_layer = Dense(1, activation='softplus', name='prediccion_final')(dense_2)

    model = Model(inputs=[input_cuadrante, input_sequence], outputs=output_layer)
    model.compile(optimizer='adam', loss=Poisson(), metrics=['mae'])
    
    return model


# --- 5. EVALUACIÓN Y CÁLCULO FINAL DEL R^2 ---

# 5.1. Predicción en escala de conteo (no normalizada)
y_pred_real_lstm = model.predict([X_cuadrante_test_seq, X_test_seq])

# 5.2. Calcular el R^2
r2_lstm = r2_score(Y_test_seq, y_pred_real_lstm.flatten())

print("\n--- 📈 EVALUACIÓN DEL MODELO LSTM ---")
print(f"El Coeficiente de Determinación (R^2) en el conjunto de prueba es: {r2_lstm:.4f}")

# --- 6. VISUALIZACIÓN DE LA COMPARACIÓN ---

# Crear DataFrame para el R^2
df_test_filtered = df_test.iloc[TIME_STEPS:].copy().reset_index(drop=True)

df_results = pd.DataFrame({
    'CUADRANTE_ID': X_cuadrante_test_seq,
    'FECHA': df_test_filtered['FECHA'].values,
    'ROBOS_REAL': Y_test_seq,
    'PREDICCION_ROBOS_REAL': y_pred_real_lstm.flatten()
})

# Filtrar para Enero 2024
MES_A_EVALUAR = 1
df_enero_2024 = df_results[
    (df_results['FECHA'].dt.month == MES_A_EVALUAR) & 
    (df_results['FECHA'].dt.year == 2024)
].copy()

df_enero_2024['PREDICCION_ROBOS_REAL'] = df_enero_2024['PREDICCION_ROBOS_REAL'].round(0).astype(int)
df_enero_2024['ROBOS_REAL'] = df_enero_2024['ROBOS_REAL'].round(0).astype(int)
df_enero_2024['ERROR (Real - Pred)'] = df_enero_2024['ROBOS_REAL'] - df_enero_2024['PREDICCION_ROBOS_REAL']
df_enero_2024 = df_enero_2024.sort_values(by='PREDICCION_ROBOS_REAL', ascending=False)

df_final = df_enero_2024[['CUADRANTE_ID', 'ROBOS_REAL', 'PREDICCION_ROBOS_REAL', 'ERROR (Real - Pred)']]

print(f"\n--- Comparación Enero 2024 (Ordenado por Predicción) ---")
print(df_final.head(20).to_markdown(index=False))

2025-11-19 17:04:52.250024: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-19 17:05:14.876986: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-19 17:05:25.393239: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.



--- COMENZANDO ENTRENAMIENTO DE LA RED NEURONAL LSTM ---
Epoch 1/100


2025-11-19 17:05:34.958107: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


395/395 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.5037 - mae: 1.1195 - val_loss: 0.7023 - val_mae: 1.0853
Epoch 2/100
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3890 - mae: 1.0330 - val_loss: 0.6190 - val_mae: 0.9660
Epoch 3/100
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3743 - mae: 1.0127 - val_loss: 0.6346 - val_mae: 0.9847
Epoch 4/100
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3704 - mae: 1.0125 - val_loss: 0.6411 - val_mae: 0.9973
Epoch 5/100
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3673 - mae: 1.0120 - val_loss: 0.6195 - val_mae: 0.9480
Epoch 6/100
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3642 - mae: 1.0060 - val_loss: 0.6346 - val_mae: 0.9851
Epoch 7/100
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3653 - mae: 1.0093 - val_loss: 0.6329 - val_mae: 0.9837
Epoch 8/100
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3629 - mae: 1.0065 - val_loss: 0.6375 - val_mae: 0.9876
Epoch 9/100
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss:

ValueError: All arrays must be of the same length